In [ ]:
import pandas as pd, matplotlib.pyplot as plt, nltk, re
from nltk.stem import WordNetLemmatizer
import functions
from collections import Counter
import numpy as np

In [ ]:
def preprocess(text, verbose=True):
    preprocessed = []
    repeat = 0
    lemmatizer = WordNetLemmatizer()
    for t, tweet in enumerate(text):
        if verbose:
            print("Samples: {:,}/{:,}".format(t+1, len(text)), end="\r", flush=True)
        tmp = tweet.lower()
        tmp = re.sub("(http(s)?://)?(www\.)?([a-zA-Z0-9])+\.(com|net|org)(/\S*)?",'urllink', tmp)
        tmp = re.sub("[0-9]{1,2}/[0-9]{1,2}/([0-9]{4}|[0-9]{2})|([0-9]{4}|[0-9]{2})/[0-9]{1,2}/[0-9]{1,2}|[0-9]{2}/[0-9]{2}", "DATE", tmp)
        tmp = re.sub("[0-9]{2}(:[0-9]{2}){1,2}( ?(am|pm|p\.m\.|a\.m\.))?", "TIME", tmp)
        tmp = re.sub("[0-9]+", " nb ", tmp)
        tmp = tmp.encode(encoding='ascii', errors='ignore').decode()
        # get rid of the non ascii characters
        tmp = re.sub("\.{2,}", "...", tmp)
        for c in ['!', '-', ',']:
            tmp = re.sub("{}+".format(c), c, tmp)
        tmp = re.sub("\?+", "?", tmp)
        tmp = re.sub("`+", "'", tmp)
        tmp = re.sub("'{2,}", "'", tmp)
        
        tmp = tmp.translate(str.maketrans(dict.fromkeys('#*+<=>@[\\]^_`{|}~'))) #removing all the other special characters
        tokens = [t if t not in ["''", "``"] else '"' for t in nltk.tokenize.word_tokenize(tmp) ]
        # tokenizing using nltk.word_tokenize. althought it transforms '"' into '``' or "''" and this is a behaviour do not want
        # so we make sure that the '"' are changed to their original form
        i=0
        while i < len(tokens):
            if re.search("([a-z]+\.+[a-z]+|\.+[a-z]+|[a-z]+\.+)", tokens[i]):
                #all the tokens containing dots are splited around dots
                splited =  re.split("\.", tokens[i])
                tmp = splited.pop(0) 
                while splited and not tmp:
                    tmp = splited.pop(0)
                tokens[i] = tmp
                j = i+1
                while splited:
                    tmp = splited.pop(0)
                    if tmp:
                        tokens.insert(j, tmp)
                        j += 1
            i += 1
                
        for i, token in enumerate(tokens):
            if re.search(r"([a-z])\1{2,}", token):
                repeat += 1
                # cleaning the words containing a letter repeated 3 times or more, using the list of the Ensglish words
                tokens[i] = functions.clean(token)
            tokens[i] = lemmatizer.lemmatize(tokens[i])
        preprocessed.append(tokens)
    if verbose:
        print("{} words with repetitions cleaned".format(repeat))
    return preprocessed

In [ ]:
data = pd.read_csv("data/train_posts.csv", header=None)

In [ ]:
preprocessed = preprocess(data[0].values)

In [ ]:
new_df = pd.DataFrame({"text" : preprocessed, "label":data[1].values})

In [ ]:
new_df.to_pickle("data/test_preprocessed.pkl")